In [23]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/gameofcode'

role = sagemaker.get_execution_role()

s3path = 's3://{}/{}/output'.format(sagemaker_session.default_bucket(), prefix)

In [24]:
!aws s3 cp ./model.tar.gz $s3path

upload: ./model.tar.gz to s3://sagemaker-us-east-2-984881703516/sagemaker/gameofcode/output


In [27]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from utils import load_model


# default content type is numpy array
NP_CONTENT_TYPE = 'application/x-npy'

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    model = load_model(model_dir)
    
    model.to(device).eval()
    print("Done loading model.")
    
    return model


def input_fn(serialized_input_data, content_type):
    print('Deserializing the input data.')
    if content_type == NP_CONTENT_TYPE:
        stream = BytesIO(serialized_input_data)
        return np.load(stream)
    raise Exception('Requested unsupported ContentType in content_type: ' + content_type)


def output_fn(prediction_output, accept):
    print('Serializing the generated output.')
    return prediction_output


def pr

In [28]:
!pygmentize serve/model.py

import torch
import torch.nn.functional as F


class DevolutionNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, output_size):
        super(DevolutionNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.output_size = output_size
        self.bn1 = torch.nn.BatchNorm1d(num_features=self.input_size)

        self.input = torch.nn.Linear(self.input_size, self.hidden_layer_sizes[0])
        self.hidden = []
        for k in range(len(self.hidden_layer_sizes) - 1):
            self.hidden.append(torch.nn.Linear(self.hidden_layer_sizes[k], self.hidden_layer_sizes[k + 1]))
        self.output = torch.nn.Linear(self.hidden_layer_sizes[-1], self.output_size)

    def forward(self, x):
        x = self.bn1(x)
        x = F.relu(self.input(x))

        for layer in self.hidden:
            x = F.relu(layer(x))
        # x = F.softmax(self.output(x), dim=1)
        x = self.output(x)
       

In [29]:
!pygmentize serve/utils.py

from model import DevolutionNetwork
import torch
import json
import os

def load_model(model_dir):
    model_info_path = os.path.join(model_dir, 'model.json')
    with open(model_info_path, 'r') as fp:
        data = json.load(fp)
        net = DevolutionNetwork(**data)
        model_path = os.path.join(model_dir, 'model.pth')
        net.load_state_dict(torch.load(model_path))
    return net


In [30]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(Predictor, self).__init__(endpoint_name, sagemaker_session, content_type='application/x-npy')

model = PyTorchModel(model_data=s3path,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=Predictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------------------------*

UnexpectedStatusException: Error hosting endpoint sagemaker-pytorch-2020-10-10-16-03-32-446: Failed. Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [21]:
import numpy as np

test = np.array([4.1000e+01, 2.0000e+00, 4.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+00,
         5.0000e+00, 4.3776e+04, 3.2220e+04, 4.5537e+01, 2.7973e+04, 3.4928e+04,
         8.8400e+03, 6.6230e+03, 8.8100e+03, 9.6590e+03, 2.8120e+03, 0.0000e+00,
         0.0000e+00, 9.0000e+04])
test

array([4.1000e+01, 2.0000e+00, 4.0000e+00, 0.0000e+00, 0.0000e+00,
       2.0000e+00, 5.0000e+00, 4.3776e+04, 3.2220e+04, 4.5537e+01,
       2.7973e+04, 3.4928e+04, 8.8400e+03, 6.6230e+03, 8.8100e+03,
       9.6590e+03, 2.8120e+03, 0.0000e+00, 0.0000e+00, 9.0000e+04])

In [49]:
predictor.predict(test)

b'0'

In [50]:
predictor.endpoint

'sagemaker-pytorch-2020-08-14-08-09-17-846'

In [51]:
#predictor.delete_endpoint()

gameofcode-deploy.ipynb
tar: lost+found: Cannot open: Permission denied
model.tar.gz
serve/
serve/requirements.txt
serve/utils.py
serve/predict.py
serve/.ipynb_checkpoints/
serve/model.py
tar: Exiting with failure status due to previous errors
